# Creating a Deployment

### Introduction

Now so far we have learned about pods in Kubernetes.  In Kubernetes, pods are the fundamental unit that we work with to organize our application.  But, as we'll see, we normally do not create pods directly. Instead we'll create a pod through a deployment.  With a deployment, we'll begin to see some of the benefits of using kubernetes.  As we'll see, with deployments, we can create multiple replicas of the same pod, update these replicas, and get automatic rollbacks if there is an error.  Ok, let's get to it.

### Why Deployments

Remember that when we manager our applications, we may need not just one pod performing a task, but multiple pods performing that exact same task.  For example, think of our pod that runs our flask api.  If our web traffic increases, that pod may get overloaded with requests, and we'll want to create multiple pods that perform the exact same task.

A **deployment** let's us *manage* an identical set of pods, and serves as a parent object to those pods.  With a deployment, we describe how many replicas of a pod we would prefer, and can update those pods as a unit.

<img src="./api-deployment-2.jpg" width="60%">

### Creating a Deployment

Ok, let's get started.  To create our deployment, we fill in information for the same top level keys as we saw in our pod yaml file.

```yaml
apiVersion:
kind:
metadata:
spec:
```

For our deployment, the object is part of the apps module, so apiVersion is `apps/v1`, and the object type is `Deployment`.  Just like previously we specify the name of the object here `api-deployment` under `metadata`.

```yaml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: api-deployment
spec:
  replicas: 2
  template:
```

The contents of the spec key looks different.  To begin with, we now use the `replicas` key to specify the number of pods we would like deployed.  And if we continue, we specify the type of deployment with the `template` key.  The template key is where we specify the details of the pods that make up the deployment.  Let's see that below.  

### Specifying the Pod

So now that we have worked through the `apiVersion`, `kind`, and `metadata` and `replicas` in a deployment, the next step is define the pod that we are replicating.  This comes under the `spec.template` key.  If we look at it below, notice that we specify our `metadata` and `spec` for the pod in precisely the same way that we did in initially creating the pod. 

```yaml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: api-deployment
spec:
  replicas: 2
  template:
    metadata:
      labels:
        component: api
    spec:
      containers:
        - name: flask-api
          image: jek2141/flask_api
          ports:
          - containerPort: 5000
```

So the keys under template, specify our pod by filling in the `metadata` and `spec` keys just like before.  

> This time around, we do not specify `kind: Pod`, as a deployment always creates a Pod. 

So we can think of our deployment file in two parts: the top level keys pertaining to the deployment, and the `spec.template` that describes the template of the pod we are deploying.

### Connecting the Deployment and the Pod

There is one last bit of work that we need to do before our deployment is complete.  We need to explicitly tell the deployment what pods it will be managing.  We do so with a `spec.selector.matchLabels` key.

```yaml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: api-deployment
spec:
  replicas: 2
  selector:
    matchLabels:
      component: api
  template:
    metadata:
      labels:
        component: api
    spec:
      containers:
        - name: flask-api
          image: jek2141/flask_api
          ports:
          - containerPort: 5000
```

The above says to find or create any pods with the label of `component: api` and then update the cluster so that they have the specified container and number of replicas.

### Creating our Deployment

Now that we've filled in the yaml file for a deployment, it's time to create it.  We can do so with our apply function.

`kubectl apply -f api-deployment.yaml`

And if we look we'll see that we are creating two different pods, each with the specified containers.

<img src="./api-deployments.png" width="80%">

We can also view the deployment itself.  

And because the deployment *manages* the collection of replica pods, we can always decrease the number of pods or change their image with a change to the deployment file.

```yaml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: api-deployment
spec:
  replicas: 1
  template:
    metadata:
      labels:
        component: api
    spec:
      containers:
        - name: flask-api
          image: jek2141/flask_api:v1
          ports:
          - containerPort: 5000
```

`kubectl apply -f api-deployment.yaml`

> Above decreasing the number of replicas from two to one, will lead kubernetes to remove one of the pods, and changing the image to `flask_api:v1` will lead kubernetes to pull down the new image and upgrade the container on the still existing pod.

### Summary

In the lifecycle of our application, we may want to have replicas of a single pod so that we can scale up certain microservices.  In this lesson, we saw how we can use a deployment to manage a collection of replica pods through a deployment.  We can use a deployment to initially create a set of replica pods, but then change the number of replicas, or change the running container.  Our config file allows to manage all of this.

```yaml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: api-deployment
spec:
  replicas: 1
  template:
    metadata:
      labels:
        component: api
    spec:
      containers:
        - name: flask-api
          image: jek2141/flask_api:v1
          ports:
          - containerPort: 5000
```

We simply need to change the specified replicas, or alter the image value, and then run `kubectl apply -f api-deployment.yaml`.  Kubernetes will then take care of updating the our deployment so it matches our specifications.

### Resources

[Pods, ReplicaSets and Deployments Blog](https://blog.macstadium.com/blog/how-to-k8s-pods-replicasets-and-deployments)